In [1]:
import os
main_dir = os.path.dirname(os.path.dirname(os.getcwd()))
os.chdir(main_dir)

import pyvdirs.dirs as dirs
import sys
sys.path.insert(0, dirs.SYSTEM_HOME)
sys.path.insert(0, os.path.join(dirs.SYSTEM_HOME, "ToyExample"))
from socket import gethostname

import builtins
import pickle
import torch
import numpy as np
import json
import matplotlib.pyplot as plt

from ToyExample.toy_example import gt, mandala_score
import pyvtools.text as vtext
import ours.utils as utils

%load_ext autoreload
%autoreload 2

In [2]:
series = "30_TestEfficiency"
classes='A'
test_n_samples=10*2**14
vmax = 10250 # 3600
device=torch.device('cuda')
gnet_path = os.path.join(dirs.MODELS_HOME, "ToyExample/00_PreTrained/Ref/iter0512.pkl")

In [3]:
data = "> 30_TestEfficiency ACIDNonInverted_073 >>> Time >>> 1279 >>> 15.866415882110596\
> 30_TestEfficiency ACIDNonInverted_073 >>> Examples >>> 4095 >>> 3342336\
> 30_TestEfficiency ACIDNonInverted_231 >>> Time >>> 1279 >>> 15.861848334471384\
> 30_TestEfficiency ACIDNonInverted_231 >>> Examples >>> 4095 >>> 3342336\
> 30_TestEfficiency ACIDNonInverted_000 >>> Time >>> 1279 >>> 15.796777415275574\
> 30_TestEfficiency ACIDNonInverted_000 >>> Examples >>> 4095 >>> 3342336\
> 30_TestEfficiency ACIDNonInverted_172 >>> Time >>> 1279 >>> 15.885940523942311 \
> 30_TestEfficiency ACIDNonInverted_172 >>> Examples >>> 4095 >>> 3342336 \
> 30_TestEfficiency ACIDNonInverted_357 >>> Time >>> 1535 >>> 16.134254491329195 \
> 30_TestEfficiency ACIDNonInverted_357 >>> Examples >>> 4095 >>> 3342336\
> 30_TestEfficiency EarlyACIDNonInverted_073 >>> Time >>> 4095 >>> 17.272693932056427\
> 30_TestEfficiency EarlyACIDNonInverted_073 >>> Examples >>> 1023 >>> 3630144\
> 30_TestEfficiency EarlyACIDNonInverted_357 >>> Time >>> 3583 >>> 16.468802710374195\
> 30_TestEfficiency EarlyACIDNonInverted_357 >>> Examples >>> 1023 >>> 3003664 \
> 30_TestEfficiency EarlyACIDNonInverted_172 >>> Time >>> 3839 >>> 16.44512361685435 \
> 30_TestEfficiency EarlyACIDNonInverted_172 >>> Examples >>> 1023 >>> 3564544 \
> 30_TestEfficiency EarlyACIDNonInverted_231 >>> Time >>> 3583 >>> 16.695520043373108 \
> 30_TestEfficiency EarlyACIDNonInverted_231 >>> Examples >>> 1023 >>> 3020064 \
> 30_TestEfficiency EarlyACIDNonInverted_000 >>> Time >>> 4095 >>> 17.21331090927124 \
> 30_TestEfficiency EarlyACIDNonInverted_000 >>> Examples >>> 1023 >>> 3561264 \
> 30_TestEfficiency Random_000 >>> Time >>> 3839 >>> 16.457821106910707 \
> 30_TestEfficiency Random_000 >>> Examples >>> 4095 >>> 3354624 \
> 30_TestEfficiency Random_172 >>> Time >>> 3839 >>> 16.855511740843454 \
> 30_TestEfficiency Random_172 >>> Examples >>> 4095 >>> 3354624 \
> 30_TestEfficiency Random_357 >>> Time >>> 3839 >>> 16.96301788886388 \
> 30_TestEfficiency Random_357 >>> Examples >>> 4095 >>> 3354624 \
> 30_TestEfficiency Random_073 >>> Time >>> 4095 >>> 16.956689266363778 \
> 30_TestEfficiency Random_073 >>> Examples >>> 4095 >>> 3354624 \
> 30_TestEfficiency Random_231 >>> Time >>> 3839 >>> 16.97756971915563 \
> 30_TestEfficiency Random_231 >>> Examples >>> 4095 >>> 3354624\
> 30_TestEfficiency NoACID_357 >>> Time >>> 3839 >>> 16.64228993654251\
> 30_TestEfficiency NoACID_357 >>> Examples >>> 767 >>> 3145728\
> 30_TestEfficiency NoACID_000 >>> Time >>> 3839 >>> 17.218669092655183 \
> 30_TestEfficiency NoACID_000 >>> Examples >>> 767 >>> 3145728 \
> 30_TestEfficiency NoACID_073 >>> Time >>> 3839 >>> 16.55296248992284 \
> 30_TestEfficiency NoACID_073 >>> Examples >>> 767 >>> 3145728 \
> 30_TestEfficiency NoACID_172 >>> Time >>> 3839 >>> 16.540579922993977 \
> 30_TestEfficiency NoACID_172 >>> Examples >>> 767 >>> 3145728 \
> 30_TestEfficiency NoACID_231 >>> Time >>> 3839 >>> 16.666315853595734 \
> 30_TestEfficiency NoACID_231 >>> Examples >>> 767 >>> 3145728 "

In [4]:
data = [d for d in data.split("> 30_TestEfficiency") if d != ""]

In [5]:
data = [d.split(" ") for d in data]
data = [[d for d in dat if d not in ["",">>>"]] for dat in data]

In [6]:
data_time = []
data_examples = []
for dat in data:
    if "Time" in dat:
        dat.remove("Time")
        data_time.append(dat)
    elif "Examples" in dat:
        dat.remove("Examples")
        data_examples.append(dat)

In [7]:
times = [float(dat[-1]) for dat in data_time]
print(np.mean(times), np.std(times))

16.573605743845302 0.4583550910060457


In [8]:
examples = [float(dat[-1]) for dat in data_examples]
print(np.mean(examples), np.std(examples))

3299656.0 166777.41602507216


In [21]:
# Unguided
gtd, gtcomps = gt(classes, device)
for dat in data_time:
    folder, epoch, time = dat
    epoch = int(epoch)
    fig_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", series, f"mandala_time_{epoch+1:04}_"+folder+".png")
    net_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", series, folder, f"iter{epoch+1:04}.pkl")

    with builtins.open(net_filepath, "rb") as f:
        net = pickle.load(f).to(device)
    ema_mandala, ema_classification = mandala_score(net, gtd, n_samples=test_n_samples, plotting=True,
                                                    save_fig=fig_filepath, show_stats=False, 
                                                    vmax=vmax, log_scale=True)
    plt.close()

In [22]:
# Unguided
gtd, gtcomps = gt(classes, device)
for dat in data_examples:
    folder, epoch, examples = dat
    epoch = int(epoch)
    fig_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", series, f"mandala_examples_{epoch+1:04}_"+folder+".png")
    net_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", series, folder, f"iter{epoch+1:04}.pkl")

    with builtins.open(net_filepath, "rb") as f:
        net = pickle.load(f).to(device)
    ema_mandala, ema_classification = mandala_score(net, gtd, n_samples=test_n_samples, plotting=True,
                                                    save_fig=fig_filepath, show_stats=False, 
                                                    vmax=vmax, log_scale=True)
    plt.close()

In [23]:
# Guided
gtd, gtcomps = gt(classes, device)
with builtins.open(gnet_path, "rb") as f:
    gnet = pickle.load(f).to(device)
for dat in data_time:
    folder, epoch, time = dat
    epoch = int(epoch)
    fig_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", series, f"mandala_time_guided_{epoch+1:04}_"+folder+".png")
    net_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", series, folder, f"iter{epoch+1:04}.pkl")

    with builtins.open(net_filepath, "rb") as f:
        net = pickle.load(f).to(device)
    ema_mandala, ema_classification = mandala_score(net, gtd, guide=gnet, guidance_weight=3,
                                                    n_samples=test_n_samples, plotting=True,
                                                    save_fig=fig_filepath, show_stats=False, 
                                                    vmax=vmax, log_scale=True)
    plt.close()

In [24]:
# Guided
gtd, gtcomps = gt(classes, device)
with builtins.open(gnet_path, "rb") as f:
    gnet = pickle.load(f).to(device)
for dat in data_examples:
    folder, epoch, examples = dat
    epoch = int(epoch)
    fig_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", series, f"mandala_examples_guided_{epoch+1:04}_"+folder+".png")
    net_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", series, folder, f"iter{epoch+1:04}.pkl")

    with builtins.open(net_filepath, "rb") as f:
        net = pickle.load(f).to(device)
    ema_mandala, ema_classification = mandala_score(net, gtd, guide=gnet, guidance_weight=3,
                                                    n_samples=test_n_samples, plotting=True,
                                                    save_fig=fig_filepath, show_stats=False,
                                                    vmax=vmax, log_scale=True)
    plt.close()